In [ ]:
import pandas as pd
import os
import sys
import zipfile
import json
from tqdm.auto import tqdm
tqdm.pandas()
import re
import pickle
util_dir = os.path.abspath('../utils')
sys.path.append(util_dir)
from utils import *

In [ ]:
file = 'csv/treasury_shoes2.txt'
tr_df = pd.read_csv(file, encoding='utf8')

In [ ]:
ids = list(tr_df['id_text'])
ids.sort()

In [ ]:
project = 'epsd2/admin/ur3'
#oracc_download([project])

In [ ]:
def parsejson(text):
    for JSONobject in text["cdl"]:
        if "cdl" in JSONobject: 
            parsejson(JSONobject)
        if "label" in JSONobject:
            meta_d["label"] = JSONobject['label']
        if "f" in JSONobject:
            lemma = JSONobject["f"]
            if "ftype" in JSONobject:
                lemma['ftype'] = JSONobject['ftype'] # this picks up YN for year name
            lemma["id_word"] = JSONobject["ref"]
            lemma['label'] = meta_d["label"]
            lemma["id_text"] = meta_d["id_text"]
            lemm_l.append(lemma)
        if "strict" in JSONobject and JSONobject["strict"] == "1":
            lemma = {key: JSONobject[key] for key in dollar_keys}
            lemma["id_word"] = JSONobject["ref"]
            lemma["id_text"] = meta_d["id_text"]
            lemm_l.append(lemma)
    return

In [ ]:
lemm_l = []
meta_d = {"label": None, "id_text": None}
dollar_keys = ["extent", "scope", "state"]
file = f"jsonzip/{project.replace('/', '-')}.zip"
try:
    z = zipfile.ZipFile(file) 
except:
    print(f"{file} does not exist or is not a proper ZIP file")
files = z.namelist()
files = [name for name in files if name[-12:-5] in ids]
for filename in tqdm(files, desc = project):
    id_text = project + filename[-13:-5] 
    meta_d["id_text"] = id_text
    #try:
    st = z.read(filename).decode('utf-8')
    data_json = json.loads(st)           
    parsejson(data_json)
    #except:
    #print(f'{id_text} is not available or not complete')
z.close()

In [ ]:
words = pd.DataFrame(lemm_l).fillna('')
keep = ['extent', 'scope', 'state', 'id_word', 'id_text', 'form', 'cf', 'gw', 'pos']
words = words[keep]

In [ ]:
findreplace = {' ' : '-', ',' : ''}
words = words.replace({'gw' : findreplace, 'sense' : findreplace}, regex=True)

In [ ]:
words['id_text'] = [i[-7:] for i in words['id_text']]
words['id_line'] = [int(i.split('.')[1]) for i in words['id_word']]

In [ ]:
proper_nouns = ['FN', 'PN', 'DN', 'AN', 'WN', 'ON', 'TN', 'MN', 'CN', 'GN']
physical_break = ['illegible', 'traces', 'missing', 'effaced']
logical_break = ['other', 'blank', 'ruling']
words['lemma'] = words["cf"] + '[' + words["gw"] + ']' + words["pos"]
words.loc[words["cf"] == "" , 'lemma'] = words['form'] + '[NA]NA'
words.loc[words["pos"] == "n" , 'lemma'] = words['form'] + '[]NU'
words["lemma"] = words.progress_apply(lambda r: f"{r['lemma'][:-5]}1]{r['pos']}" 
                            if r["pos"] in proper_nouns else r['lemma'], axis=1)
words.loc[words["state"].isin(logical_break), 'lemma'] = "break_logical"
words.loc[words["state"].isin(physical_break), 'lemma'] = "break_physical"
words.head(10)

In [ ]:
normdf = pd.read_csv('Normalized/drehem_norm_names.csv', encoding='utf8')
normdf

In [ ]:
letters = r'a-zḫĝŋṣšṭA-ZḪĜŊṢŠṬ'
lettersX = re.compile(fr'(?<=[{letters}])x') # X preceded by a letter
lettersNo = re.compile(fr'[{letters}](\d+|x)') # any sequence of digits, or X, preceded by a letter
xv = re.compile(fr'[\w]+x') #this matches a sequence of word signs (letters) and/or flags, followed by X
xvalues = {'nagx' : '₃', 'nigarx' : '', 'nemurx' : '₂', 'pešx' : '₁₄', 'urubx' : '', 
        'tubax' : '₄', 'niginx' : '₈', 'šux' : '₁₄', 
        'alx' : 'ₓ(|NUN.LAGAR|)' , 'bulugx' : 'ₓ(|ŠIM×KUŠU₂|)', 'dagx' : 'ₓ(KWU844)', 
        'durux' : 'ₓ(|IGI.DIB|)', 'durunx' : 'ₓ(|KU.KU)', 
        'gigirx' : 'ₓ(|LAGAB×MU|)', 'giparx' : 'ₓ(KISAL)', 'girx' : 'ₓ(GI)', 
        'gišbunx' : 'ₓ(|KI.BI|)', 'gurx' : 'ₓ(|ŠE.KIN|)', 
        'hirinx' : 'ₓ(KWU318)', 'kurunx' : 'ₓ(|DIN.BI|)',
        'mu-kux' : 'ₓ(DU)', 'munsubx' : 'ₓ(|PA.GU₂×NUN|)',  
        'sagx' : 'ₓ(|ŠE.KIN|)', 'subx' : 'ₓ(|DU.DU|)', 
        'sullimx' : 'ₓ(EN)', 'šaganx' : 'ₓ(|GA×AN.GAN|)', 
        'ulušinx' : 'ₓ(|BI.ZIZ₂|)', 'zabalamx' : 'ₓ(|MUŠ₃.TE.AB@g|)', 
        'zahx' : 'ₓ(ŠEŠ)', 'zahdax' : 'ₓ(|DUN.NE.TUR|)',  
        'zix' : 'ₓ(IGI@g)'}

In [ ]:
def ogsl_v(text):
    # 1. deal with unambiguous x-values, listed in the dictionary xvalues.
    ogsl_valid = re.sub(xv, lambda m: m.group()[:-1] + xvalues.get(m.group().lower(), 'x'), text)
    return ogsl_valid

In [ ]:
normdf["atftr"] = normdf['transliteration'].progress_map(ogsl_v) 

In [ ]:
oracc_download(['ogsl'])

In [ ]:
equiv = {'ANŠE' : 'GIR₃', 
        'DUR₂' : 'KU', 
        'NAM₂' : 'TUG₂', 
        'TIL' : 'BAD', 
        'NI₂' : 'IM',
        'ŠAR₂' : 'HI', 
        }
w = re.compile(r'\w+') # replace whole words only - do not replace TILLA with BADLA.
           # but do replace |SAL.ANŠE| with |SAL.GIR₃|

In [ ]:
def parsejson(data_json):
    for key, value in data_json["signs"].items():
        key = re.sub(w, lambda m: equiv.get(m.group(), m.group()), key)
        if "values" in value:
            for n in value["values"]:
                d2[n] = key
    return

In [ ]:
d2 = {}  # this empty dictionary is filled by the parsejson() function, called in this cell.
file = "jsonzip/ogsl.zip"
z = zipfile.ZipFile(file) 
filename = "ogsl/ogsl-sl.json"
signlist = z.read(filename).decode('utf-8')
data_json = json.loads(signlist)                # make it into a json object (essentially a dictionary)
parsejson(data_json)  
with open('output/ogsl_dict.p', 'wb') as p:
    pickle.dump(d2, p)  

In [ ]:
separators = ['{', '}', '-']
separators2 = ['.', '+', '|']  # used in compound signs
#operators = ['&', '%', '@', '×']
flags = "][?<>⸢⸣⌈⌉*/" # note that ! is omitted from flags, because it is dealt with separately
table = str.maketrans(dict.fromkeys(flags))

In [ ]:
def signnames(translit):  
    """This function takes a string of transliterated cuneiform text and translates that string into a string of
    sign names, separated by spaces. In order to work it needs the variables separators, separators2, and table defined above. The variable table
    is used by the translate() method to translate all flags (except for !) to None. The function also needs a dictionary, called d2, that has as
    keys sign readings and sign names as corresponding values. In case a key is not found, the sign reading is replaced by itself."""
    signnames_l = []
    translit = translit.translate(table).lower()  # remove flags, half brackets, square brackets.
    translit = translit.replace('...', 'x')
    for s in separators: # split transliteration line into signs   
        translit = translit.replace(s, ' ').strip()
    s_l = translit.split() # s_l is a list that contains the sequence of transliterated signs without separators or flags
    s_l = [d2.get(sign, sign) for sign in s_l] # replace each transliterated sign with its sign name.
    # Now take care of some special situations: signs with qualifiers, compound signs.
    for sign in s_l:
        if '!' in sign: # corrected sign, as in ka!(SAG), get only the corrected reading.
            sign = sign.split('!(')[0]
            sign = sign.replace('!', '') # remove remaining exclamtion marks
        elif sign[-1] == ')' and '(' in sign: # qualified sign, as in ziₓ(SIG₇) - get only the qualifier
            sign = sign.split('(')[1][:-1]
        if '×' in sign: #compound. Compound like |KA×NINDA| to be replaced by |KA×GAR|
            sign_l = sign.replace('|', '').split('×')
            #replace individual signs of the compound by OGSL names
            sign_l = [d2.get(sign, sign) for sign in sign_l] 
            # if user enters |KA*EŠ| this is transformed to ['KA', '|U.U.U|']. The pipes around U.U.U must be replaced by brackets
            sign_l = [f'({sign[1:-1]})' if len(sign) > 1 and sign[0] == '|' else sign for sign in sign_l]
            sign = f"|{'×'.join(sign_l)}|"  #put the sign together again with enclosing pipes.
        elif '.' in sign or '+' in sign: # using elif, so that compounds like |UD×(U.U.U)| are not further analyzed.
            for s in separators2:
                sign = sign.replace(s, ' ').strip() 
            sign_l = sign.split()  # compound sign split into multiple signs
            sign_l = [d2.get(sign, sign) for sign in sign_l]
            signnames_l.extend(sign_l)
            continue
        sign = d2.get(sign, sign)
        signnames_l.append(sign)
    # add space before and after each line so that each sign representation is enclosed in spaces
    signnames = f" {' '.join(signnames_l).upper()} " 
    return signnames

In [ ]:
normdf["sign_names"] = normdf["atftr"].progress_map(signnames)

In [ ]:
normd2 = dict(zip(normdf['sign_names'], normdf['normalization']))

In [ ]:
words['sign_names'] = words['form'].progress_map(signnames)

In [ ]:
words['name_norm'] = words.sign_names.progress_apply(lambda x: normd2.get(x, ''))

In [ ]:
words.loc[(words.pos == 'PN') & (words.name_norm == '')]

In [ ]:
failed = words.loc[(words.pos == 'PN') & (words.name_norm == ''), ['id_word', 'lemma', 'form']]

In [ ]:
failed

In [ ]:
failed.to_csv('Normalized/corrections.csv', encoding='utf8', index=False)

In [ ]:
normdf = normdf[['transliteration', 'normalization', 'remarks']]
normdf = normdf.fillna('')

In [ ]:
anchor = '<a href="http://build-oracc.museum.upenn.edu/epsd2/admin/ur3/{}", target="_blank">{}</a>'
failed2 = failed.copy()
failed2['id_word'] = [anchor.format(val,val) for val in failed['id_word']]
failed2.style